<a href="https://colab.research.google.com/github/misbakhanum/Emotion_detection/blob/main/emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving speech..zip to speech..zip


In [8]:
import zipfile
import os

# Path to the uploaded file (replace 'your_file.zip' with the actual file name)
zip_file = '/content/speech..zip'

# Specify the directory where you want to extract the files
extract_dir = '/content/my'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the extracted files
extracted_files = os.listdir(extract_dir)
print('Extracted files:', extracted_files)


Extracted files: ['speech']


In [9]:
import numpy as np
import os
import glob
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import librosa
import soundfile

# Emotions in the RAVDESS dataset
emotions = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad', '05': 'angry',
    '06': 'fearful', '07': 'disgust', '08': 'surprised'
}

# Emotions to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

# Function to extract features from an audio file
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        result = np.array([])

        if chroma or mel:
            stft = np.abs(librosa.stft(X))

        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))

        if chroma:
            chroma_features = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma_features))

        if mel:
            mel_features = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel_features))

    return result


In [10]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []

    # Path to the dataset folder (update this path if needed)
    dataset_path = "/content/my/speech/Actor_22"

    # Loop through each .wav file
    for file in glob.glob(os.path.join(dataset_path, "*.wav")):
        file_name = os.path.basename(file)

        # Extract the emotion from the file name (assumes format like 'Actor-01-01-01-01.wav')
        emotion_code = file_name.split("-")[2]  # Adjust if your filenames have a different structure

        # Get the emotion from the code
        emotion = emotions.get(emotion_code, None)
        if emotion is None:
            continue  # Skip files that don't match the expected emotion code

        # Only consider certain emotions
        if emotion not in observed_emotions:
            continue

        # Extract features
        feature = extract_feature(file)
        x.append(feature)
        y.append(emotion)

    # Split the dataset into train and test
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

# Print the number of samples in train and test sets
print(f"Training set size: {len(x_train)}")
print(f"Test set size: {len(x_test)}")


Training set size: 24
Test set size: 8


In [26]:
# Encode the labels
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train_encoded)

# Predict on the test set (using the encoded test labels)
y_pred_encoded = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Model accuracy: {accuracy * 100:.2f}%")

# Save the trained model as 'modelForPrediction1.sav'
model_filename = 'modelForPrediction1.sav'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)
print(f"Model saved as {model_filename}")

# Calculate the F1 Score for different classes (use encoded labels)
f1 = f1_score(y_test_encoded, y_pred_encoded, average=None)
print("F1 Score for each class:", f1)

# Convert the predicted labels back to original labels using encoder
y_pred = encoder.inverse_transform(y_pred_encoded)
y_test_decoded = encoder.inverse_transform(y_test_encoded)

# Create a Pandas DataFrame with Actual and Predicted labels
import pandas as pd
df = pd.DataFrame({'Actual': y_test_decoded, 'Predicted': y_pred})

# Display the first 20 rows of the DataFrame
print(df.head(20))


Model accuracy: 75.00%
Model saved as modelForPrediction1.sav
F1 Score for each class: [1.         0.5        0.66666667]
    Actual Predicted
0     calm      calm
1  fearful     happy
2  fearful   fearful
3    happy     happy
4     calm      calm
5     calm      calm
6  fearful     happy
7    happy     happy


In [27]:
x_train

array([[-5.60218872e+02,  3.61590576e+01, -1.05001612e+01, ...,
         7.41095719e-05,  6.96825664e-05,  2.71470999e-05],
       [-6.92916199e+02,  3.84976234e+01,  4.37359333e+00, ...,
         1.28995480e-05,  8.91466152e-06,  5.99452414e-06],
       [-6.34491638e+02,  4.20777969e+01, -8.21499729e+00, ...,
         7.87678655e-05,  3.30158000e-05,  3.87367327e-05],
       ...,
       [-6.58869690e+02,  4.60602150e+01, -3.26962924e+00, ...,
         3.69369409e-05,  2.88143656e-05,  2.11176666e-05],
       [-5.51950928e+02,  2.73531380e+01, -1.98856792e+01, ...,
         2.99765874e-04,  1.54527312e-04,  7.70317492e-05],
       [-5.35934204e+02,  3.13018532e+01, -1.84415741e+01, ...,
         1.90475781e-04,  1.63476987e-04,  6.51773225e-05]])

In [28]:
print((x_train.shape[0], x_test.shape[0]))

(24, 8)


In [29]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [30]:
y_pred=model.predict(x_test)
y_pred

array([0, 3, 2, 3, 0, 0, 3, 3])

In [34]:
import os

directory = "/content/my/speech/Actor_22"
files = os.listdir(directory)
print("Files in directory:", files)

Files in directory: ['03-01-06-02-01-01-22.wav', '03-01-03-01-02-01-22.wav', '03-01-04-02-01-02-22.wav', '03-01-07-01-02-01-22.wav', '03-01-03-02-01-02-22.wav', '03-01-02-02-01-01-22.wav', '03-01-02-01-01-02-22.wav', '03-01-07-02-02-02-22.wav', '03-01-06-01-01-01-22.wav', '03-01-05-02-01-02-22.wav', '03-01-04-01-01-02-22.wav', '03-01-08-01-02-01-22.wav', '03-01-04-01-02-01-22.wav', '03-01-06-02-01-02-22.wav', '03-01-06-02-02-01-22.wav', '03-01-03-02-02-01-22.wav', '03-01-05-01-02-02-22.wav', '03-01-08-01-01-01-22.wav', '03-01-02-01-02-02-22.wav', '03-01-08-02-02-02-22.wav', '03-01-08-02-01-01-22.wav', '03-01-02-01-02-01-22.wav', '03-01-05-02-01-01-22.wav', '03-01-03-01-01-02-22.wav', '03-01-08-02-01-02-22.wav', '03-01-02-02-02-01-22.wav', '03-01-03-02-01-01-22.wav', '03-01-07-02-01-02-22.wav', '03-01-03-01-01-01-22.wav', '03-01-05-01-02-01-22.wav', '03-01-05-02-02-02-22.wav', '03-01-04-01-02-02-22.wav', '03-01-04-01-01-01-22.wav', '03-01-01-01-01-01-22.wav', '03-01-04-02-02-01-22.wav',

In [35]:
import pickle
import numpy as np
import os
import glob

# Load the pre-trained model from the file
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# Path to the folder containing audio files
folder_path = "/content/my/speech/Actor_22"

# Iterate through all .wav files in the folder
for audio_file_path in glob.glob(os.path.join(folder_path, "*.wav")):
    try:
        # Extract features from the audio file
        feature = extract_feature(audio_file_path, mfcc=True, chroma=True, mel=True)

        # Reshape the feature array for prediction
        feature = feature.reshape(1, -1)

        # Make a prediction using the loaded model
        prediction = loaded_model.predict(feature)
        print(f"File: {os.path.basename(audio_file_path)} - Predicted emotion: {prediction[0]}")

    except Exception as e:
        print(f"An error occurred with file {audio_file_path}: {e}")

File: 03-01-06-02-01-01-22.wav - Predicted emotion: 2
File: 03-01-03-01-02-01-22.wav - Predicted emotion: 3
File: 03-01-04-02-01-02-22.wav - Predicted emotion: 3
File: 03-01-07-01-02-01-22.wav - Predicted emotion: 1
File: 03-01-03-02-01-02-22.wav - Predicted emotion: 3
File: 03-01-02-02-01-01-22.wav - Predicted emotion: 0
File: 03-01-02-01-01-02-22.wav - Predicted emotion: 0
File: 03-01-07-02-02-02-22.wav - Predicted emotion: 1
File: 03-01-06-01-01-01-22.wav - Predicted emotion: 3
File: 03-01-05-02-01-02-22.wav - Predicted emotion: 3
File: 03-01-04-01-01-02-22.wav - Predicted emotion: 0
File: 03-01-08-01-02-01-22.wav - Predicted emotion: 2
File: 03-01-04-01-02-01-22.wav - Predicted emotion: 0
File: 03-01-06-02-01-02-22.wav - Predicted emotion: 2
File: 03-01-06-02-02-01-22.wav - Predicted emotion: 2
File: 03-01-03-02-02-01-22.wav - Predicted emotion: 3
File: 03-01-05-01-02-02-22.wav - Predicted emotion: 1
File: 03-01-08-01-01-01-22.wav - Predicted emotion: 1
File: 03-01-02-01-02-02-22.w

In [36]:
# Save the trained model as 'modelForPrediction1.sav'
model_filename = 'modelForPrediction1.sav'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)
print(f"Model saved as {model_filename}")

# Save the LabelEncoder as 'label_encoder.sav'
encoder_filename = 'label_encoder.sav'
with open(encoder_filename, 'wb') as encoder_file:
    pickle.dump(encoder, encoder_file)
print(f"Label Encoder saved as {encoder_filename}")

Model saved as modelForPrediction1.sav
Label Encoder saved as label_encoder.sav


In [38]:
# Load the pre-trained model and LabelEncoder
model_filename = 'modelForPrediction1.sav'
encoder_filename = 'label_encoder.sav'

# Load the model
try:
    with open(model_filename, 'rb') as model_file:
        loaded_model = pickle.load(model_file)
    print(f"Model loaded successfully from {model_filename}")
except FileNotFoundError:
    print(f"Error: Model file {model_filename} not found.")
    loaded_model = None

# Load the LabelEncoder
try:
    with open(encoder_filename, 'rb') as encoder_file:
        loaded_encoder = pickle.load(encoder_file)
    print(f"Label Encoder loaded successfully from {encoder_filename}")
except FileNotFoundError:
    print(f"Error: Encoder file {encoder_filename} not found.")
    loaded_encoder = None

# Check if both model and encoder are loaded successfully
if loaded_model is None or loaded_encoder is None:
    print("Cannot proceed without both model and label encoder.")
else:
    # Path to the folder containing new audio files for prediction
    folder_path = "/content/my/speech/Actor_22"

    # Iterate through all .wav files in the folder
    for audio_file_path in glob.glob(os.path.join(folder_path, "*.wav")):
        try:
            # Extract features from the audio file
            feature = extract_feature(audio_file_path, mfcc=True, chroma=True, mel=True)

            # Check if feature extraction was successful
            if feature is not None:
                # Reshape the feature array for prediction
                feature = feature.reshape(1, -1)

                # Make a prediction using the loaded model
                prediction = loaded_model.predict(feature)

                # Convert numeric prediction to human-readable emotion
                emotion = loaded_encoder.inverse_transform(prediction)

                # Output the file path and predicted emotion
                print(f"{audio_file_path}: [{emotion[0]}]")
            else:
                print(f"Feature extraction failed for {audio_file_path}. Skipping file.")
        except Exception as e:
            print(f"Error processing {audio_file_path}: {e}")

Model loaded successfully from modelForPrediction1.sav
Label Encoder loaded successfully from label_encoder.sav
/content/my/speech/Actor_22/03-01-06-02-01-01-22.wav: [fearful]
/content/my/speech/Actor_22/03-01-03-01-02-01-22.wav: [happy]
/content/my/speech/Actor_22/03-01-04-02-01-02-22.wav: [happy]
/content/my/speech/Actor_22/03-01-07-01-02-01-22.wav: [disgust]
/content/my/speech/Actor_22/03-01-03-02-01-02-22.wav: [happy]
/content/my/speech/Actor_22/03-01-02-02-01-01-22.wav: [calm]
/content/my/speech/Actor_22/03-01-02-01-01-02-22.wav: [calm]
/content/my/speech/Actor_22/03-01-07-02-02-02-22.wav: [disgust]
/content/my/speech/Actor_22/03-01-06-01-01-01-22.wav: [happy]
/content/my/speech/Actor_22/03-01-05-02-01-02-22.wav: [happy]
/content/my/speech/Actor_22/03-01-04-01-01-02-22.wav: [calm]
/content/my/speech/Actor_22/03-01-08-01-02-01-22.wav: [fearful]
/content/my/speech/Actor_22/03-01-04-01-02-01-22.wav: [calm]
/content/my/speech/Actor_22/03-01-06-02-01-02-22.wav: [fearful]
/content/my/sp